In [64]:
import sys
sys.path.append('/Users/spindicate/Documents/programming/zooscraper/')

In [65]:
import django
django.setup()

In [66]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [67]:
import pandas as pd
import numpy as np
import itertools

from django.db import transaction

In [68]:
BEG_AGES = [i * 5 for i in range(int(85 / 5))]
END_AGES = [i * 5 + 4 for i in range(int(85 / 5))]
age_ranges = list(zip(BEG_AGES, END_AGES))
AGE_COLS = ['A' + str(age_range[0]).zfill(2) + '_' + str(age_range[1]).zfill(2) + 'B' for age_range in age_ranges]
AGE_UPPERS = ['A' + str(age) + 'PLUSB' for age in [65, 70, 75, 80, 85]]

In [69]:
RANGES = {    
    'AGE_UNDERS': {
        'ranges': ['A' + str(age).zfill(2) + 'UNDERB' for age in [9, 14, 19, 24, 29, 34]],
        'range_slice': (1, 3),
        'fix_position': 0,
        'fix_direction': 'beg',
    },
    'SCHOOL_GOERS': {
        'ranges': ['A05_' + str(age) + 'B' for age in [14, 19, 24, 29, 34]],
        'range_slice': (-3, -1),
        'fix_position': 1,
        'fix_direction': 'beg',
    },
    'Y_MILLS': {
        'ranges': ['A15_' + str(age) + 'B' for age in [24, 29, 34]],
        'range_slice': (-3, -1),
        'fix_position': 3,
        'fix_direction': 'beg',
    },
    'MILLS': {
        'ranges': ['A20_' + str(age) + 'B' for age in [29, 34]],
        'range_slice': (-3, -1),
        'fix_position': 4,
        'fix_direction': 'beg',
    },
    'MID': {
        'ranges': ['A' + str(age) + '_54B' for age in [35, 40, 45]],
        'range_slice': (1, 3),
        'fix_position': -7,
        'fix_direction': 'end',
    },
    'MID_PLUS': {
        'ranges': ['A' + str(age) + '_64B' for age in [35, 40, 45]],
        'range_slice': (1, 3),
        'fix_position': -5,
        'fix_direction': 'end',
    }
}

In [70]:
def find_col(check):
    locations = [i for i, x in enumerate(AGE_COLS) if check in x]

    if len(locations) > 1 or len(locations) < 1:
        raise ValueError
    
    return locations.pop()

def col_slicer(var_position, fix_position, fix='end'):
    if fix == 'end':
        return var_position, fix_position + 1
    elif fix == 'beg':
        return fix_position, var_position + 1

def get_cols(df):
    age_dict = {age_col: df[age_col].sum() for age_col in AGE_COLS}
    for age_range in AGE_UPPERS:
        age_dict[age_range] = df[age_range].sum()

    for range_type, values in RANGES.items():
        for age_range in values['ranges']:
            location = find_col(age_range[values['range_slice'][0]:values['range_slice'][1]])
            i, j = col_slicer(location, values['fix_position'], values['fix_direction'])
            cols = AGE_COLS[i:j]
            age_dict[age_range] = df[cols].sum().sum()
    
    if (df['COUNTRYNM'] == 'Italy').all():
        population = df[[*AGE_COLS, 'A75PLUSB']].sum().sum()
    else:
        population = df[[*AGE_COLS, 'A85PLUSB']].sum().sum()
    
    LAND_A_KM = df['LAND_A_KM'].sum()
    land_density = population / LAND_A_KM
    
    cities = []
    for i, df_city in df.groupby('NAME2'):
        df_city = df_city.copy(deep=True)
        
        if not df_city.empty:
        
            if (df['COUNTRYNM'] == 'Italy').all():
                city_pop = df_city[[*AGE_COLS, 'A75PLUSB']].sum().sum()
            else:
                city_pop = df_city[[*AGE_COLS, 'A85PLUSB']].sum().sum()

            city_land = df_city['LAND_A_KM'].sum()
            city_dens = city_pop / city_land

            cities.append((i, city_pop, city_dens))

    cities_sorted = sorted(cities, key=lambda tup: tup[1], reverse=True)

    if len(cities) > 0:
        city_dens = cities_sorted[0][2]
    else:
        city_dens = land_density
            
    return {
        'population': population,
        'LAND_A_KM': LAND_A_KM,
        'land_density': land_density,
         **age_dict,
        'city_dens': city_dens,
    }

In [80]:
def get_exUS_pops(df):
    weirds = ['GLP', 'MTQ', 'MAF', 'MYT']
    with transaction.atomic():
#         for region in Region.objects.filter(country_key__alpha3='BRA').exclude(name='Brazil'):
        for region in Region.objects.filter(country_key__alpha3__in=weirds):
            print (region.country_key.alpha3)
            if region.name == region.country:
                if region.country in df['COUNTRYNM'].unique():
                    pop_data = get_cols(df[(df['COUNTRYNM'] == region.country)])
                elif region.country_alt in df['COUNTRYNM'].unique():
                    pop_data = get_cols(df[(df['COUNTRYNM'] == region.country_alt)])
            elif region.name == 'Hong Kong':
                pop_data = get_cols(df[(df['COUNTRYNM'] == region.country_alt)])
            elif region.country_key.alpha3 in weirds:
                print ('here')
                pop_data = get_cols(df[(df['ISOALPHA'] == region.country_key.alpha3)])
            else:
                if region.name in df['NAME1'].unique():
                    pop_data = get_cols(df[(df['NAME1'] == region.name)])
                elif region.name_alt in df['NAME1'].unique():
                    pop_data = get_cols(df[(df['NAME1'] == region.name_alt)])
                elif region.name in df['NAME2'].unique():
                    pop_data = get_cols(df[(df['NAME2'] == region.name)])
                elif region.name_alt in df['NAME2'].unique():
                    pop_data = get_cols(df[(df['NAME2'] == region.name_alt)])            

            print (region.name)
            Region.objects.filter(id=region.id).update(**pop_data)
            print (pop_data)

            if not pop_data['population']:
                print ('*********** No data: ', region.name)

In [72]:
def get_US_pops(df):
    with transaction.atomic():
        for region in Region.objects.filter(country='USA'):
            if region.name in df['NAME1'].unique():
                pop_data = get_cols(df[(df['NAME1'] == region.name)])
            elif region.name_alt in df['NAME1'].unique():
                pop_data = get_cols(df[(df['NAME1'] == region.name_alt)])
            elif region.name in df['NAME2'].unique():
                pop_data = get_cols(df[(df['NAME2'] == region.name)])
            elif region.name_alt in df['NAME2'].unique():
                pop_data = get_cols(df[(df['NAME2'] == region.name_alt)])

            Region.objects.filter(id=region.id).update(**pop_data)

            print (region.name)
            if not pop_data['population']:
                print ('*********** No data: ', region.name)

In [6]:
path = '/Users/spindicate/Documents/docs/covid19/pops/gpw-v4-admin-unit-center-points-population-estimates-rev11_global_csv/gpw_v4_admin_unit_center_points_population_estimates_rev11_global.csv'
df_exUS = pd.read_csv(path)

# Need to Amend Chinese names
df_exUS.NAME1 = df_exUS.NAME1.str.title()
df_exUS.NAME2 = df_exUS.NAME2.str.title()
df_exUS.NAME3 = df_exUS.NAME3.str.title()

In [73]:
weirds = ['GLP', 'MTQ', 'MAF', 'MYT']

In [74]:
df_exUS = df_exUS[df_exUS.ISOALPHA.isin(weirds)]

In [81]:
get_exUS_pops(df_exUS)

GLP
here
Guadeloupe
{'population': 403519.74242565164, 'LAND_A_KM': 1659.2715979295467, 'land_density': 243.19089347950452, 'A00_04B': 27494.93541758968, 'A05_09B': 30115.67538500101, 'A10_14B': 31631.81403876457, 'A15_19B': 31632.090220425987, 'A20_24B': 21082.748496108026, 'A25_29B': 19552.148136639407, 'A30_34B': 23616.75941579287, 'A35_39B': 29950.88050016116, 'A40_44B': 32028.564857344056, 'A45_49B': 31745.205624411275, 'A50_54B': 26686.012133238204, 'A55_59B': 24210.457414889293, 'A60_64B': 20256.694655243908, 'A65_69B': 15737.552341606779, 'A70_74B': 13196.202241628405, 'A75_79B': 10039.650855696933, 'A80_84B': 7310.467736663381, 'A65PLUSB': 53515.75613004215, 'A70PLUSB': 37778.20378843536, 'A75PLUSB': 24582.001546806965, 'A80PLUSB': 14542.350691110028, 'A85PLUSB': 7231.882954446648, 'A09UNDERB': 57610.61080259069, 'A14UNDERB': 89242.42484135526, 'A19UNDERB': 120874.51506178125, 'A24UNDERB': 141957.26355788927, 'A29UNDERB': 161509.41169452868, 'A34UNDERB': 185126.17111032156, 'A

In [50]:
path = '/Users/spindicate/Documents/docs/covid19/pops/gpw-v4-admin-unit-center-points-population-estimates-rev11_global_csv/'
files = [file for file in os.listdir(path) if 'usa' in file]
dfs = [pd.read_csv(path + file) for file in files]
df_us = pd.concat(dfs)
get_US_pops(df_us)

Alabama
Puerto Rico
Guam
Virgin Islands
Wyoming
Alaska
Arkansas
Delaware
Idaho
Maine
Mississippi
Montana
New Mexico
North Dakota
South Dakota
West Virginia
Michigan
Vermont
Georgia
Colorado
Florida
New Jersey
Oregon
Texas
Illinois
Pennsylvania
Iowa
Maryland
North Carolina
Washington
New York
California
Massachusetts
Diamond Princess
Oklahoma
Arizona
Connecticut
Minnesota
Virginia
New Hampshire
Hawaii
Nevada
Indiana
Kentucky
District of Columbia
Missouri
Louisiana
Ohio
Wisconsin
Kansas
Utah
Tennessee
South Carolina
Nebraska
Rhode Island
Northern Mariana Islands
United States Virgin Islands
